93784 seconds and 566 milliseconds


In [96]:
from pickle import FALSE
#Code submitted by Joel Vinas
import requests                 #Used to import file directly from the web
import io                       #Used to import file directly from the web
import pandas as pd             #Used to work with data
from datetime import datetime   #Used to determine Now()
from datetime import timedelta  #Used to sum timedeltas
import random                   #Used to generate a random number

#Import the file directly from the web
url = 'https://github.com/joelvinas/COMP-SCI_5501/blob/aa4391ae9d3f2cb73e89f3b1fd5bc86bfe34fe2b/Assignment2/Data/words_alpha.txt'
response = requests.get(url)

response.raise_for_status()
if response.status_code == 200:
  dictionary = response.text.splitlines()
else:
  print(f"Error: {response.status_code}")
  exit()

class Treatment:
  iteration = 0
  random_word = ""
  starttime = ""
  endtime = ""
  def __init__(self, iteration, random_word, starttime):
    self.iteration = iteration
    self.random_word = random_word
    self.starttime = starttime
    self.comparisons = 0

  def duration(self):
    return self.endtime - self.starttime

class SearchTools:
  linear_results = []
  sentinel_results = []
  binary_results = []
  binary_sorted_results = []
  ternary_results = []
  ternary_sorted_results = []

  def SearchDuration(self, results):
    curSearchDuration = timedelta(0)
    for curIteration in results:
      curSearchDuration += curIteration.duration()
    SearchMilliseconds = int((curSearchDuration.total_seconds() - int(curSearchDuration.total_seconds())) * 1000)
    return SearchMilliseconds

  def SearchComparisons(self, results):
    totalComparisons = 0
    for curIteration in results:
      totalComparisons += curIteration.comparisons
    return totalComparisons


  def getRandomWord(self, dictionary):
    random_int = random.randint(1, len(dictionary))
    return dictionary[random_int]

  def linear_search(self, index, dictionary, random_word):
    starttime = datetime.now()
    comparisons = 1
    wordIsFound = False
    curTreatment = Treatment(index, random_word, starttime)
    for curWord in dictionary:
      if curWord == random_word:
        curTreatment.endtime = datetime.now()
        wordIsFound = True
        break
      else:
        comparisons += 1
    if not wordIsFound:
      curTreatment.endtime = datetime.now()
    curTreatment.comparisons = comparisons
    return curTreatment

  def sentinel_search(self, index, dictionary, random_word):
    starttime = datetime.now()
    comparisons = 1
    wordIsFound = False
    sentinel_dictionary = dictionary.copy()
    sentinel_dictionary.append(random_word)
    curTreatment = Treatment(index, random_word, starttime)

    for curWord in sentinel_dictionary:
      if curWord == random_word:
        curTreatment.endtime = datetime.now()
        wordIsFound = True
        break
      else:
        comparisons += 1
    #If the word is never found
    if not wordIsFound:
      curTreatment.endtime = datetime.now()
    curTreatment.comparisons = comparisons
    return curTreatment

  def binary_search(self, index, dictionary, random_word, presorted=False):
      starttime = datetime.now()
      comparisons = 1
      wordIsFound = False
      binary_dictionary = dictionary.copy()
      if not presorted:
        binary_dictionary.sort()
      curTreatment = Treatment(index, random_word, starttime)

      begining = 0
      ending = len(binary_dictionary) - 1

      while begining <= ending and not wordIsFound:
        mid = (begining + ending) // 2
        if binary_dictionary[mid] == random_word:
          wordIsFound = True
          curTreatment.endtime = datetime.now()
        else:
          if random_word < binary_dictionary[mid]:
            ending = mid - 1
            comparisons += 1
          else:
            begining = mid + 1
            comparisons += 1
    #If the word is never found
      if not wordIsFound:
        curTreatment.endtime = datetime.now()
      curTreatment.comparisons = comparisons
      return curTreatment

  def ternary_search(self, index, dictionary, random_word, presorted=False):
      starttime = datetime.now()
      comparisons = 1
      wordIsFound = False
      ternary_dictionary = dictionary.copy()
      if not presorted:
        ternary_dictionary.sort()
      curTreatment = Treatment(index, random_word, starttime)

      begining = 0
      ending = len(ternary_dictionary) - 1

      while begining <= ending and not wordIsFound:
        mid1 = begining + (ending - begining) // 3
        mid2 = ending - (ending - begining) // 3
        #Is the word at either of the two midpoints?
        if ternary_dictionary[mid1] == random_word:
          wordIsFound = True
          curTreatment.endtime = datetime.now()
        if ternary_dictionary[mid2] == random_word:
          wordIsFound = True
          curTreatment.endtime = datetime.now()

        if not wordIsFound:
          comparisons += 1
          if random_word < ternary_dictionary[mid1]:
            #Word is in the first 3rd of the range
            ending = mid1 - 1
          elif random_word > ternary_dictionary[mid2]:
            #Word is in the last 3rd of the range
            begining = mid2 + 1
          else:
            #Word is in the second 3rd of the range
            begining = mid1 + 1
            ending = mid2 - 1

    #If the word is never found
      if not wordIsFound:
        curTreatment.endtime = datetime.now()
      curTreatment.comparisons = comparisons
      return curTreatment


  def getDataframe(self, method_name, results):
    column_names = ["method", "iteration", "random_word", "starttime", "endtime", "duration", "comparisons"]
    df = pd.DataFrame(columns = column_names)

    for curIteration in results:
      new_row = [method_name, curIteration.iteration, curIteration.random_word, curIteration.starttime, curIteration.endtime, curIteration.duration(), curIteration.comparisons]
      df.loc[len(df)] = new_row
    return df

#Start Operations
SearchTools = SearchTools()
TotalOperations = 20

#Initialize Random Word list
randWordList = []
for randWord in range(TotalOperations):
  randWordList.append(SearchTools.getRandomWord(dictionary))

# Linear Search
for iteration in range(TotalOperations):
  random_word = randWordList[iteration]
  curTreatment = SearchTools.linear_search(iteration, dictionary, random_word)
  SearchTools.linear_results.append(curTreatment)

# Sentinel Search
for iteration in range(TotalOperations):
  random_word = randWordList[iteration]
  curTreatment = SearchTools.sentinel_search(iteration, dictionary, random_word)
  SearchTools.sentinel_results.append(curTreatment)

# Binary Search (Unsorted)
for iteration in range(TotalOperations):
  random_word = randWordList[iteration]
  curTreatment = SearchTools.binary_search(iteration, dictionary, random_word)
  SearchTools.binary_results.append(curTreatment)

# Ternary Search (Unsorted)
for iteration in range(TotalOperations):
  random_word = randWordList[iteration]
  curTreatment = SearchTools.ternary_search(iteration, dictionary, random_word)
  SearchTools.ternary_results.append(curTreatment)

#Presorted Binary/Ternary Searches
sorted_dictionary = dictionary.copy()
sorted_dictionary.sort()

# Binary Search (Presorted)
for iteration in range(TotalOperations):
  random_word = randWordList[iteration]
  curTreatment = SearchTools.binary_search(iteration, sorted_dictionary, random_word, True)
  SearchTools.binary_sorted_results.append(curTreatment)

# Ternary Search (Presorted)
for iteration in range(TotalOperations):
  random_word = randWordList[iteration]
  curTreatment = SearchTools.ternary_search(iteration, dictionary, random_word, True)
  SearchTools.ternary_sorted_results.append(curTreatment)

#5: Compare the number of comparisons made in each treatment
print("Question 1 Task 5: Search Algorithms completed for {TotalOperations} samples:")
print(f"Linear Search: {SearchTools.SearchDuration(SearchTools.linear_results)} milliseconds, {format(SearchTools.SearchComparisons(SearchTools.linear_results),',')} comparisons")
print(f"Sentinel Search: {SearchTools.SearchDuration(SearchTools.sentinel_results)} milliseconds, {format(SearchTools.SearchComparisons(SearchTools.sentinel_results),',')} comparisons")
print(f"Binary Search (Unsorted): {SearchTools.SearchDuration(SearchTools.binary_results)} milliseconds, {format(SearchTools.SearchComparisons(SearchTools.binary_results),',')} comparisons")
print(f"Ternary Search (Unsorted): {SearchTools.SearchDuration(SearchTools.ternary_results)} milliseconds, {format(SearchTools.SearchComparisons(SearchTools.ternary_results),',')} comparisons")
print(f"Binary Search (Presorted): {SearchTools.SearchDuration(SearchTools.binary_sorted_results)} milliseconds, {format(SearchTools.SearchComparisons(SearchTools.binary_sorted_results),',')} comparisons")
print(f"Ternary Search (Presorted): {SearchTools.SearchDuration(SearchTools.ternary_sorted_results)} milliseconds, {format(SearchTools.SearchComparisons(SearchTools.ternary_sorted_results),',')} comparisons")


df_all_results = pd.DataFrame()
df_all_results = pd.concat([df_all_results, SearchTools.getDataframe("Linear Search", SearchTools.linear_results)], ignore_index=True)
df_all_results = pd.concat([df_all_results, SearchTools.getDataframe("Sentinel Search", SearchTools.sentinel_results)], ignore_index=True)
df_all_results = pd.concat([df_all_results, SearchTools.getDataframe("Binary Search (Unsorted)", SearchTools.binary_results)], ignore_index=True)
df_all_results = pd.concat([df_all_results, SearchTools.getDataframe("Ternary Search (Unsorted)", SearchTools.ternary_results)], ignore_index=True)
df_all_results = pd.concat([df_all_results, SearchTools.getDataframe("Binary Search (Presorted)", SearchTools.binary_sorted_results)], ignore_index=True)
df_all_results = pd.concat([df_all_results, SearchTools.getDataframe("Ternary Search (Presorted)", SearchTools.ternary_sorted_results)], ignore_index=True)

df_all_results

#Question 1 Task 7  Show the comparative analysis for each treatment (for step 5—6) individually in form of:
#Question 1 Task 7i. Tabulated data
#Question 1 Task 7ii. Line Chart

#Question 1 Task 8  Show the comparative analysis as average time for each algorithm in form of:
#Question 1 Task 8  i. Tabulated Data
#Question 1 Task 8  ii. Bar Charts
#Question 1 Task 8  iii. Exaplain your observations.

Search Algorithms completed for {TotalOperations} samples:
Linear Search: 287 milliseconds, 3,441,047 comparisons
Sentinel Search: 231 milliseconds, 3,441,047 comparisons
Binary Search (Unsorted): 523 milliseconds, 355 comparisons
Ternary Search (Unsorted): 509 milliseconds, 229 comparisons
Binary Search (Presorted): 40 milliseconds, 355 comparisons
Ternary Search (Presorted): 34 milliseconds, 236 comparisons


,method,iteration,random_word,starttime,endtime,duration,comparisons
0,Linear Search,0,diplasion,2025-10-08 15:05:32.971981,2025-10-08 15:05:32.980888,0 days 00:00:00.008907,86501
1,Linear Search,1,semester,2025-10-08 15:05:32.980906,2025-10-08 15:05:33.008086,0 days 00:00:00.027180,283695
2,Linear Search,2,crossbill,2025-10-08 15:05:33.008107,2025-10-08 15:05:33.016107,0 days 00:00:00.008000,73779
3,Linear Search,3,outpath,2025-10-08 15:05:33.016123,2025-10-08 15:05:33.037818,0 days 00:00:00.021695,217057
4,Linear Search,4,stringene,2025-10-08 15:05:33.037835,2025-10-08 15:05:33.067437,0 days 00:00:00.029602,306333
...,...,...,...,...,...,...,...
115,Ternary Search (Presorted),15,renovater,2025-10-08 15:05:34.850150,2025-10-08 15:05:34.851706,0 days 00:00:00.001556,9
116,Ternary Search (Presorted),16,diddering,2025-10-08 15:05:34.853300,2025-10-08 15:05:34.854870,0 days 00:00:00.001570,12
117,Ternary Search (Presorted),17,adsorbing,2025-10-08 15:05:34.856473,2025-10-08 15:05:34.857945,0 days 00:00:00.001472,11
118,Ternary Search (Presorted),18,unintrudingly,2025-10-08 15:05:34.859529,2025-10-08 15:05:34.861077,0 days 00:00:00.001548,12
